In [12]:
import numpy as np
import pandas as pd
import subprocess as sub
import matplotlib.pyplot as plt
import time

## step 1. Generate sample set by metropolis method

In [44]:
start = time.time()
sub.run(['g++', 'step1_plaquette_srch.cpp', '-o', 'step1_s', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step1_s'])
print("step1 finished :", time.time()-start)

#Data of Energy&nn in fileout.txt
f = pd.read_csv('fileout_srch.txt', sep = ' ', header = 0)
temp = f.temp[0]
J0 = np.array(f.ene)
J1 = np.array(f.nn)
J2 = np.array(f.nnn)
J3 = np.array(f.nnnn)
num = len(J0)
nth = int(f.nth[0]) #How much consideration of nearest neighbors
Jmat = np.array([J1, J2, J3]).T
target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
print("T =", temp, " Energy, J =", Jlist, " error =", err)

#Data of coefficient in filein.txt
with open("filein_srch.txt", "w") as file:
    file.write('%d\n' %nth)
    file.write('%.10g\n' %temp)
    for i in range (nth + 1):
        data = Jlist[i]
        file.write('%.10g\n' % data)

step1 finished : 411.9806430339813
T = 4.493  Energy, J = [ 1.45676625  1.09832968 -0.02216503]  error = [31389.86428501]


## step 2. optimize J, regenerate sample set by wolff method

In [45]:
sub.run(['g++', 'step2_wolff_srch.cpp', '-o', 'step2_s', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
for i in range (15):
    start = time.time()
    sub.run(['./step2_s'])
    print("step2 :", time.time()-start)

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout_srch.txt', sep = ' ', header = 0)
    temp = f.temp[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein_srch.txt", "w") as file:
        file.write('%d\n' %nth)
        if temp>2.5: file.write('%.10g\n' %temp)
        else: file.write('%.10g\n' %(temp+0.2))
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)

step2 : 132.774005651474
T = 4.293  Energy, J = [ 1.73330109  1.10414861 -0.02362926]  error = [30392.86482094]
step2 : 116.34569549560547
T = 4.093  Energy, J = [ 1.98791392  1.11219397 -0.02955134]  error = [29506.56387647]
step2 : 97.24413466453552
T = 3.8930000000000002  Energy, J = [ 2.16068862  1.11446663 -0.02815554]  error = [28153.41544157]
step2 : 85.864187002182
T = 3.693  Energy, J = [ 2.53626614  1.12433717 -0.03459302]  error = [26623.29898305]
step2 : 66.69420623779297
T = 3.4930000000000003  Energy, J = [ 2.84548077  1.13017355 -0.03597988]  error = [25384.86475632]
step2 : 49.96368145942688
T = 3.293  Energy, J = [ 3.64478282  1.14567176 -0.04388756]  error = [22343.60541864]
step2 : 30.726420879364014
T = 3.093  Energy, J = [ 4.27566731  1.15747434 -0.05009235]  error = [20524.45179224]
step2 : 13.264950513839722
T = 2.8930000000000002  Energy, J = [ 5.29331166  1.17599038 -0.06083904]  error = [17073.06567647]
step2 : 7.354280233383179
T = 2.693  Energy, J = [ 6.7039

## step 3. Compare magnetization

In [ ]:
start = time.time()
sub.run(['g++', 'step3_plot_srch.cpp', '-o', 'step3_s', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step3_s'])
print("step3 :", time.time()-start)

In [ ]:
cols = ['m', 'ms', 'e', 'sp_h']
p = pd.read_csv('../../python/txtfiles-5/p10.txt', sep=' ', header=0)
fit = pd.read_csv('plot_srch.txt', sep = ' ', header = 1)
print("Plaquette & Fitted of L=10")

for j in range (4):
    plt.plot(p.temperature, p[cols[j]], 'r.', markersize = 10, markerfacecolor = None, linestyle = 'None', label = 'original')
    plt.plot(fit.temperature, fit[cols[j]], '.', markersize = 3, linestyle = 'None', label = 'fitted')
    plt.xlabel('T', fontsize=14)
    plt.ylabel(cols[j], fontsize=14)
    plt.xlim(1.5, 3.5)
    plt.legend()
    plt.show()